In [21]:
pip install -qq langchain_huggingface langchain_community faiss-cpu transformers accelerate pypdf langchain_experimental ragas

SyntaxError: invalid syntax (ipython-input-2771672486.py, line 2)

In [23]:
pip install -qq langchain_google_genai

PDF RAG

In [ ]:
import glob
from langchain_community.document_loaders import PyPDFLoader
folder_path = "my_pdfs/*.pdf"
pdf_files = glob.glob(folder_path)  # Collect all PDFs

all_pdfdocs = []
for file_path in pdf_files:
    loader = PyPDFLoader(file_path)
    pdfdocs = loader.load()
    all_pdfdocs.extend(pdfdocs)  #split all_pdfdocs to chunks

In [3]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "llmfacts.pdf"
loader = PyPDFLoader(file_path)
pdfdoc = loader.load()

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

pdfdocchunks = RecursiveCharacterTextSplitter(chunk_size=300,chunk_overlap=30).split_documents(pdfdoc)
# Check number of chunks
print(f"Total chunkspdf: {len(pdfdoc)}")
print(f"Total chunkspdf: {len(pdfdocchunks)}")

Total chunkspdf: 11
Total chunkspdf: 136


In [4]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
#embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
semantic_splitter = SemanticChunker(embeddings)
pdfdocchunkssy = semantic_splitter.split_documents(pdfdoc)

print(f"Total semantic chunks: {len(pdfdocchunkssy)}")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Total semantic chunks: 42


In [ ]:
print(f"Total semantic chunks: {len(pdfdocchunkssy)}")
print(f"recurssive ########## {pdfdocchunks[10].page_content}" )
print(f"symatic #############{pdfdocchunkssy[10].page_content}")
print(f"symatic #############{pdfdocchunkssy[11].page_content}")

Total semantic chunks: 42
recurssive ########## 13. E. Coli is a rod-shaped gut-residing bacteria from the Escherichia genus.
14. Antibiotics are often used against bacteria but the rising problem of antibiotic resistance has lead 
to the issue of “superbugs”. Penicillin is the ﬁrst commercial antibiotic, derived from a type of mold
symatic #############45. Calories is the unit used to count energy intake by food. 46. The respiratory system, which consists of organs like the lungs, nose, mouth, and trachea is 
responsible for keeping us breathing. This is done through taking in oxygen and removing carbon 
dioxide. 47. The nervous system is responsible for all of our conscious and unconscious thoughts, actions, and 
perceptions, and functions as the control center of our body. The central nervous system consists 
of two parts, the brain and the spinal cord. Neurons are cells that make up the spinal cord and 
brain.
symatic #############48. The cardiovascular system, which is also known 

In [6]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
vectordb_pdf = FAISS.from_documents(pdfdocchunkssy,HuggingFaceEmbeddings())  #used smaller pdf chunks
retriever_pdf = vectordb_pdf.as_retriever(search_kwargs={"k": 3})     #parm for top 5 docs   search_kwargs={"k": 5}

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
pdfdocchunks[5]
#pdfdoc[10].page_content[:]
#webdoc[0].metadata
#webdoc[0].page_content

Document(metadata={'producer': 'macOS Version 12.0.1 (Build 21A559) Quartz PDFContext', 'creator': 'Pages', 'creationdate': "D:20211106083141Z00'00'", 'title': '200 Science Facts to Know!', 'moddate': "D:20211106083141Z00'00'", 'source': 'llmfacts.pdf', 'total_pages': 11, 'page': 0, 'page_label': '1'}, page_content='This process requires the doubling of a cell’s DNA so that each resulting cell has a copy of the \ngenetic information. \n8. While mitosis produces two identical daughter cells, in order for sexual reproduction to occur,')

WebBase Loader RAG

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
webpage = WebBaseLoader("https://en.wikipedia.org/wiki/Rahul_Dravid")
webdoc=webpage.load()

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
webdocchunks = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(webdoc)

In [ ]:
webdocchunks[3].page_content

Document(metadata={'source': 'https://en.wikipedia.org/wiki/Rahul_Dravid', 'title': 'Rahul Dravid - Wikipedia', 'language': 'en'}, page_content='Appearance\nmove to sidebar\nhide\n\n\n\n\n\n\n\n\n\n\n\nFrom Wikipedia, the free encyclopedia\n\n\nIndian coach and former cricketer (born 11 January 1973)\n\n\nRahul DravidDravid in 2024Personal informationFull\xa0nameRahul Sharad DravidBorn (1973-01-11) 11 January 1973 (age\xa052)Indore, Madhya Pradesh, IndiaNicknameThe Wall, Mr Dependable[1]Height1.80\xa0m (5\xa0ft 11\xa0in)BattingRight-handedBowlingRight arm off breakRoleBatsmanWebsitewww.rahuldravid.comInternational information\nNational sideIndia (1996–2012)Test debut (cap\xa0207)20 June 1996\xa0v\xa0EnglandLast Test24 January 2012\xa0v\xa0AustraliaODI debut (cap\xa095)3 April 1996\xa0v\xa0Sri LankaLast ODI16 September 2011\xa0v\xa0EnglandODI shirt no.19Only T20I (cap\xa038)31 August 2011\xa0v\xa0EnglandT20I shirt no.19\nDomestic team information\n\nYearsTeam1990–2012Karnataka2000Kent20

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
vectordb_web = FAISS.from_documents(webdocchunks,HuggingFaceEmbeddings())
retriever_web = vectordb_web.as_retriever(search_kwargs={"k": 1})     #parm for top 5 docs   search_kwargs={"k": 5} search_type="similarity"

Retiever

In [ ]:
results[0]   # return ppage content of 1st retrieval context

Document(id='916bbd8c-7b8e-4392-ad53-c156f629875e', metadata={'producer': 'macOS Version 12.0.1 (Build 21A559) Quartz PDFContext', 'creator': 'Pages', 'creationdate': "D:20211106083141Z00'00'", 'title': '200 Science Facts to Know!', 'moddate': "D:20211106083141Z00'00'", 'source': 'llmfacts.pdf', 'total_pages': 11, 'page': 10, 'page_label': '11'}, page_content='192. Erwin Schrödinger is an Austrian physician who demonstrated superposition using his namesake \ncat.\n193. Marie Curie is a Polish scientist who coined the term “radioactivity" and also discovered radium \nwith her husband, Pierre.')

In [7]:
query = "which country is Marie Curie from?"

results = retriever_pdf.invoke(query)    # old results = retriver.get_relevant_documents(query)
#context = "\n".join([doc.page_content for doc in results])

print(f"""Only use the context to answer the question provide citation. if you do not find answer then reply "Data not found in context"

Question {query}

Context={results[0].page_content}
      """)

Only use the context to answer the question provide citation. if you do not find answer then reply "Data not found in context"

Question which country is Marie Curie from?

Context=192. Erwin Schrödinger is an Austrian physician who demonstrated superposition using his namesake 
cat. 193. Marie Curie is a Polish scientist who coined the term “radioactivity" and also discovered radium 
with her husband, Pierre. 194. Leonhard Euler is a Swiss mathematician from the 18th century who names the transcendental 
number e. He discovered the number which serves as the base of the natural logarithm. 195. Johannes Kepler is an astronomer who devised three laws of planetary motion. 196. Archimedes is a Greek mathematician of Syracuse who discovered the principle of buoyancy. 197. Louis Pasteur is a scientist who names a process used to keep milk sanitary. He was the creator of 
the rabies and anthrax vaccine invented a process where mild heat is used to remove pathogens. 198. Albert Einstein is a 

In [ ]:
query = "which country is Marie Curie from?"

results = retriever_pdf.invoke(query)
for i, doc in enumerate(results, 1):
    print(f"\nResult {i}:")
    #print("Source:", doc.metadata.get("source"))
    #print("Title:", doc.metadata.get("title"))
    print("Content snippet:", doc.page_content.replace("\n",""))  #print("Content snippet:", doc.page_content[:200], "...")


Result 1:
Content snippet: 192. Erwin Schrödinger is an Austrian physician who demonstrated superposition using his namesake cat.193. Marie Curie is a Polish scientist who coined the term “radioactivity" and also discovered radium with her husband, Pierre.

Result 2:
Content snippet: 196. Archimedes is a Greek mathematician of Syracuse who discovered the principle of buoyancy.197. Louis Pasteur is a scientist who names a process used to keep milk sanitary. He was the creator of the rabies and anthrax vaccine invented a process where mild heat is used to remove pathogens.

Result 3:
Content snippet: 189. Thomas Edison is an American inventor, who developed the most popular incandescent light bulb.190. Alan Turing is an English mathematician, who names a test which can determine if a computer possesses strong artiﬁcial intelligence.


In [8]:
from huggingface_hub import login
from google.colab import userdata

# Paste your Hugging Face token here
login(userdata.get('HF_TOKEN'))

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM,AutoModelForSeq2SeqLM

model_id = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

#direct invoke
#input_text = "what is the capital of france"
#input_ids = tokenizer(input_text, return_tensors="pt")
#outputs = model.generate(**input_ids)
#print(tokenizer.decode(outputs[0]))

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [10]:
from transformers import pipeline
from langchain_huggingface import HuggingFacePipeline
import torch

pipe = pipeline(
    task="text2text-generation",
    model=model,
    tokenizer=tokenizer,
    device=-1,  # CPU
    max_new_tokens=200
)

# Wrap in LangChain
llm = HuggingFacePipeline(pipeline=pipe)

Device set to use cpu


In [ ]:
llm.invoke(""" You are a SQL data analyst. You help write sql code and visualize the results in charts.

        Write a SQL query to answer the following question with only the tables/columns provided in metadata:
        Only return the SQL query starting with the Keyword "SQLQuery": Do not explain.
        Suggest a visualization to display above Result, starting with the Keyword "Chart:".Do not explain. choose one of (bar, pie, line, table)
        Add comment in query field selection to indicate 2 coordinates for the chart. use ("-- x coordinate", "-- y coordinate") Do not rename the query result field names.
        Sample:
        SQLQuery:
        SELECT d.department_name as Department             -- x coordinate (Department)
                , count(e.employee_id) as Employees        -- y coordinate (Employee Count)
        FROM employee e
        JOIN department d on e.department_id=d.department_id

        Chart: bar chart


        Metadata = Table: products; Columns: product_id	,product_name ,category	,brand	,price;
                    Table: sales; Columns: sale_id	,product_id	,store_id	,quantity	,sale_date;
                    Table: stores; Columns:store_id	,store_name	,city	,state	,region;
        Question: show volume of sales, revenue by product category """)

'SELECT T1.product_id , T1.product_name , T1.category , T1.brand , T1.price FROM products AS T1 JOIN sales AS T2 ON T1.product_id = T2.product_id JOIN store AS T3 ON T1.store_id = T3.store_id JOIN sales AS T4 ON T1.store_id = T4.store_id JOIN sales AS T5 ON T1.store_id = T5.store_id JOIN sales AS T6 ON T1.store_id = T4.store_id JOIN sales AS T6 ON T1.store_id = T5.store_id JOIN sales AS T6 ON T1.store_id = T4.store_id JOIN sales AS T6 ON T1.store_id'

In [36]:
from huggingface_hub import login
from google.colab import userdata

api_key = userdata.get("GAI")
import os
os.environ["GOOGLE_API_KEY"] = api_key

from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.0)
resp = llm.invoke(""" You are a SQL data analyst. You help write sql code and visualize the results in charts.

        Write a SQL query to answer the following question with only the tables/columns provided in metadata:
        Only return the SQL query starting with the Keyword "SQLQuery": Do not explain.
        Suggest a visualization to display above Result, starting with the Keyword "Chart:".Do not explain. choose one of (bar, pie, line, table)
        Add comment in query field selection to indicate 2 coordinates for the chart. use ("-- x coordinate", "-- y coordinate") Do not rename the query result field names.
        Sample:
        SQLQuery:
        SELECT d.department_name as Department             -- x coordinate (Department)
                , count(e.employee_id) as Employees        -- y coordinate (Employee Count)
        FROM employee e
        JOIN department d on e.department_id=d.department_id

        Chart: bar chart


        Metadata = Table: products; Columns: product_id	,product_name ,category	,brand	,price;
                    Table: sales; Columns: sale_id	,product_id	,store_id	,quantity	,sale_date;
                    Table: stores; Columns:store_id	,store_name	,city	,state	,region;
        Question: show the percentage distribution of sales by category in 2025""")
print(resp.content)

SQLQuery:
SELECT
    p.category AS Category,                                 -- x coordinate (Category)
    CAST(SUM(s.quantity) AS REAL) * 100.0 / SUM(SUM(s.quantity)) OVER () AS Percentage -- y coordinate (Percentage)
FROM
    sales s
JOIN
    products p ON s.product_id = p.product_id
WHERE
    STRFTIME('%Y', s.sale_date) = '2025'
GROUP BY
    p.category
ORDER BY
    Percentage DESC;

Chart: pie chart


## runable chain

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "llmfacts.pdf"
loader = PyPDFLoader(file_path)
pdfdoc = loader.load()

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

pdfdocchunks = RecursiveCharacterTextSplitter(chunk_size=300,chunk_overlap=30).split_documents(pdfdoc)
# Check number of chunks
print(f"Total chunkspdf: {len(pdfdoc)}")
print(f"Total chunkspdf: {len(pdfdocchunks)}")

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
vectordb_pdf = FAISS.from_documents(pdfdocchunks,HuggingFaceEmbeddings())  #used smaller pdf chunks
retriever_pdf = vectordb_pdf.as_retriever(search_kwargs={"k": 3})     #parm for top 5 docs   search_kwargs={"k": 5}

In [13]:
def retrieved_context(retrieved_chunks):
    context = "\n".join([doc.page_content for doc in retrieved_chunks])  #return concated text from all chunks returned.
    #context = retrieved_chunks[0].page_content   # return ppage content of 1st retrieval context
    return context

In [14]:
from langchain.schema.runnable import RunnableParallel,RunnablePassthrough,RunnableLambda

parallel_chain = RunnableParallel({
    'Context': retriever_pdf|RunnableLambda(retrieved_context),
    'Question':RunnablePassthrough()
})

In [15]:
from langchain.prompts import PromptTemplate
template = """Only use the context to answer the question provide citation. if you do not find answer then reply "Data not found in context"

Question: {Question}

Context :{Context}"""
prompt = PromptTemplate(template=template, input_variables=["Question","Context"])

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM,AutoModelForSeq2SeqLM

model_id = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

In [ ]:
from transformers import pipeline
from langchain_huggingface import HuggingFacePipeline
import torch

pipe = pipeline(
    task="text2text-generation",
    model=model,
    tokenizer=tokenizer,
    device=-1,  # CPU
    max_new_tokens=200
)

# Wrap in LangChain
llm = HuggingFacePipeline(pipeline=pipe)

Device set to use cpu


In [16]:
from langchain.schema import StrOutputParser
main_chain = parallel_chain|prompt|llm| StrOutputParser()

In [19]:
Q = "what are Cnidaria"
print(parallel_chain.invoke(Q))
print(main_chain.invoke(Q))

{'Context': 'Arthropods are characterized by a segmented body, a chitinous exoskeleton, and paired-jointed \nappendages. Arthropod appendages include antennae, mouthparts, and legs. Examples of \narthropods include crustaceans, insects, arachnids, chilopods, and diplopods. 43. Cnidarians are characterized by their radial symmetry and presence of a specialized cell called a \ncnidocyte that is used for predation and defense. Examples of cnidarians include jellyﬁsh, sea \nanemones, and coral. 44.\n33. Phylum is the next classiﬁcation after Kingdom and some important phyla include chordata \n(organisms with a notochord), Porifera (sponges),\xa0 Cnidaria (jellyﬁsh, corals, anemones), \nNematoda (roundworms), and Arthropoda (arachnids, crustaceans, insects). 34. Non-vascular plants, examples of which include mosses, hornworts, liverworts, and some forms of \nalgae, are characterized by their lack of vascular tissue (a tissue that helps transport nutrients) and \nstructures like the root, le

1. Evaluation
* Ragas
  *   faithfullness - is the answer grounded in retrieval context
  *   answer_relevancy -
  *   context_precission - how much of retrieved context is actually useful
  *    context_recall - did we retrieve all necessary info
* LangSmith

2. Indexing
  *    During Ingestion
          * preprocess to correct issues in data when applicable
*   Text Splitting
    *   recurssivetext splitting - can creae chuncks  in middle of sentence
    *   SemanticTextSplitting - to split at end of paragraph/sentence
*   Vector Store
    *    FIASS -basic
    *    Use cloud based like Pine Cone

3. Pre-retrieval
*   Re write user Query/ Question using LLM
*   Multi query generation
    * use multiple retrieval to capture all perspective. improves performance
  * Domain Aware routing
      * use multi RAG to query based on domain of question

4. During retrieval
  *    MMR search statregy
  *    Hybrid rerieval
        *  use both semantic search and Keyword search as well
  * Reranking
      * use llm to rerank the the context

5. Post retrieval
  * Contextual COmpression
      * remove the text that is not relevat - reduce token size to llm to save cost

6. Augmentation
  *    prompt Templating -with examples
  *    Answer grownding
       *  request llm to only answer based on context and not halucinate
  *    context window optimizton

6. Generation
  *    answer with citation
  *    Gaurd railing
       * avoid unwanted answers from llm

7. System Design
  *    Multimodal
  *    Agentic
  *    Memory based
       *   personalized answer based on historical questions and llm replies